In [1]:
# импортируем библиотеки, классы и функции
import pandas as pd
import numpy as np
from sklearn.multiclass import (OneVsRestClassifier, 
                                OneVsOneClassifier,
                                OutputCodeClassifier)
from sklearn.preprocessing import (StandardScaler,
                                   LabelEncoder)
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.base import (clone,
                          MetaEstimatorMixin, 
                          ClassifierMixin, 
                          BaseEstimator)
from sklearn.model_selection import train_test_split
from sklearn.utils.metaestimators import _safe_split
from sklearn.metrics import (roc_auc_score, 
                             pairwise_distances)
from sklearn.utils.fixes import delayed
from joblib import Parallel
import warnings

# отключаем экспоненциальное представление
np.set_printoptions(precision=None, suppress=True)

In [2]:
# пример 3 классов
y_trn = np.array(['apple', 'pear', 'apple', 
                  'orange', 'pear', 'apple'])
y_trn

array(['apple', 'pear', 'apple', 'orange', 'pear', 'apple'], dtype='<U6')

In [3]:
# строковые метки преобразовываем в целочисленные
le = LabelEncoder()
y_trn = le.fit_transform(y_trn)
y_trn

array([0, 2, 0, 1, 2, 0])

In [4]:
# загружаем набор
otto = pd.read_csv('Data/ottogroup_train.csv')

# удаляем id из набора
otto.drop('id', axis=1, inplace=True)

# формируем массив меток и массив признаков,
# преобразуем в массивы NumPy
X_otto = otto.drop('target', axis=1).values
y_otto = otto['target'].values

# смотрим метки классов
np.unique(y_otto)

array(['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9'], dtype=object)

In [5]:
# создание экземпляра класса LabelEncoder
le = LabelEncoder()
# строковые метки преобразовываем в целочисленные
y_otto = le.fit_transform(y_otto)
np.unique(y_otto)

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [6]:
# разбиваем набор на обучающую и тестовую выборки 
X_otto_train, X_otto_test, y_otto_train, y_otto_test = train_test_split(
    X_otto, y_otto, random_state=42)

In [7]:
# создаем конвейер - экземпляр класса Pipeline
inh_lr_pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('logreg', LogisticRegression(solver='newton-cg', 
                                  multi_class='multinomial'))
])

# применяем LogisticRegression для
# многоклассовой классификации
# естественным образом
inh_lr_pipe.fit(X_otto_train, y_otto_train);

In [8]:
# вычисляем вероятности классов зависимой
# переменной для тестовой выборки
inh_lr_proba = np.round(
    inh_lr_pipe.predict_proba(X_otto_test), 3)
inh_lr_proba[:5]

array([[0.025, 0.514, 0.135, 0.06 , 0.09 , 0.03 , 0.084, 0.044, 0.019],
       [0.013, 0.059, 0.067, 0.002, 0.   , 0.098, 0.748, 0.001, 0.011],
       [0.004, 0.007, 0.002, 0.   , 0.   , 0.934, 0.037, 0.009, 0.006],
       [0.003, 0.001, 0.   , 0.748, 0.   , 0.219, 0.015, 0.   , 0.013],
       [0.   , 0.   , 0.   , 0.   , 0.   , 1.   , 0.   , 0.   , 0.   ]])

In [9]:
# вычисляем значения решающей функции 
# для тестовой выборки
inh_lr_dec_func = np.round(
    inh_lr_pipe.decision_function(X_otto_test), 3)
inh_lr_dec_func[:5]

array([[ -0.942,   2.076,   0.74 ,  -0.076,   0.337,  -0.776,   0.268,
         -0.387,  -1.24 ],
       [  1.088,   2.591,   2.717,  -0.648, -13.425,   3.091,   5.123,
         -1.447,   0.91 ],
       [ -0.326,   0.404,  -0.835,  -2.458,  -4.933,   5.258,   2.042,
          0.653,   0.195],
       [  0.403,  -1.058,  -2.34 ,   5.903,  -9.782,   4.674,   2.017,
         -1.661,   1.845],
       [  0.204,  -4.149,   0.794,  -7.618, -15.39 ,  17.632,   1.869,
          7.978,  -1.32 ]])

In [10]:
# вычисляем прогнозы для тестовой выборки
inh_lr_pred = inh_lr_pipe.predict(X_otto_test)
inh_lr_pred[:5]

array([1, 6, 5, 3, 5])

In [11]:
# создаем конвейер - экземпляр класса Pipeline
ovr_lr_pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('logreg', LogisticRegression(solver='newton-cg', 
                                  multi_class='ovr'))
])

# применяем LogisticRegression для
# многоклассовой классификации
# по схеме one-vs-rest
ovr_lr_pipe.fit(X_otto_train, y_otto_train);

In [12]:
# вычисляем вероятности классов зависимой
# переменной для тестовой выборки
ovr_lr_proba = np.round(
    ovr_lr_pipe.predict_proba(X_otto_test), 3)
ovr_lr_proba[:5]

array([[0.066, 0.42 , 0.137, 0.039, 0.111, 0.057, 0.083, 0.066, 0.021],
       [0.019, 0.103, 0.13 , 0.008, 0.   , 0.045, 0.655, 0.001, 0.038],
       [0.009, 0.061, 0.02 , 0.002, 0.   , 0.844, 0.046, 0.009, 0.008],
       [0.008, 0.008, 0.006, 0.447, 0.   , 0.456, 0.033, 0.   , 0.04 ],
       [0.   , 0.   , 0.008, 0.   , 0.   , 0.99 , 0.   , 0.002, 0.   ]])

In [13]:
# вычисляем значения решающей функции 
# для тестовой выборки
ovr_lr_dec_func = np.round(
    ovr_lr_pipe.decision_function(X_otto_test), 3)
ovr_lr_dec_func[:5]

array([[ -3.023,  -0.858,  -2.231,  -3.555,  -2.466,  -3.165,  -2.768,
         -3.019,  -4.201],
       [ -3.913,  -2.134,  -1.868,  -4.742, -17.881,  -3.017,   0.717,
         -7.213,  -3.2  ],
       [ -4.639,  -2.605,  -3.758,  -5.887,  -8.113,   2.842,  -2.907,
         -4.638,  -4.732],
       [ -4.709,  -4.74 ,  -4.968,  -0.086, -14.566,  -0.049,  -3.299,
         -8.045,  -3.097],
       [-10.845,  -9.92 ,  -4.786, -14.167, -22.513,  10.257,  -9.421,
         -6.303, -13.133]])

In [14]:
# вычисляем прогнозы для тестовой выборки
ovr_lr_pred = ovr_lr_pipe.predict(X_otto_test)
ovr_lr_pred[:5]

array([1, 6, 5, 5, 5])

In [15]:
# создаем конвейер - экземпляр класса Pipeline
lr_pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('logreg', LogisticRegression(solver='newton-cg'))
])

# создаем экземпляр класса OneVsRestClassifier
lr_ovr_classifier = OneVsRestClassifier(lr_pipe)
# применяем LogisticRegression для многоклассовой 
# классификации по схеме one-vs-rest через
# класс OneVsRestClassifier
lr_ovr_classifier.fit(X_otto_train, y_otto_train);

In [16]:
# вычисляем вероятности классов зависимой
# переменной для тестовой выборки
lr_ovr_classifier_proba = np.round(
    lr_ovr_classifier.predict_proba(X_otto_test), 3)
lr_ovr_classifier_proba[:5]

array([[0.066, 0.42 , 0.137, 0.039, 0.111, 0.057, 0.083, 0.066, 0.021],
       [0.019, 0.103, 0.13 , 0.008, 0.   , 0.045, 0.655, 0.001, 0.038],
       [0.009, 0.061, 0.02 , 0.002, 0.   , 0.844, 0.046, 0.009, 0.008],
       [0.008, 0.008, 0.006, 0.447, 0.   , 0.456, 0.033, 0.   , 0.04 ],
       [0.   , 0.   , 0.008, 0.   , 0.   , 0.99 , 0.   , 0.002, 0.   ]])

In [17]:
# вычисляем значения решающей функции 
# для тестовой выборки
lr_ovr_classifier_dec_func = np.round(
    lr_ovr_classifier.decision_function(X_otto_test), 3)
lr_ovr_classifier_dec_func[:5]

array([[ -3.023,  -0.858,  -2.231,  -3.555,  -2.466,  -3.165,  -2.768,
         -3.019,  -4.201],
       [ -3.913,  -2.134,  -1.868,  -4.742, -17.881,  -3.017,   0.717,
         -7.213,  -3.2  ],
       [ -4.639,  -2.605,  -3.758,  -5.887,  -8.113,   2.842,  -2.907,
         -4.638,  -4.732],
       [ -4.709,  -4.74 ,  -4.968,  -0.086, -14.566,  -0.049,  -3.299,
         -8.045,  -3.097],
       [-10.845,  -9.92 ,  -4.786, -14.167, -22.513,  10.257,  -9.421,
         -6.303, -13.133]])

In [18]:
# вычисляем прогнозы для тестовой выборки
lr_ovr_classifier_pred = lr_ovr_classifier.predict(X_otto_test)
lr_ovr_classifier_pred[:5]

array([1, 6, 5, 5, 5])

In [19]:
# создаем экземпляр класса OneVsOneClassifier
lr_ovo_classifier = OneVsOneClassifier(lr_pipe)
# применяем LogisticRegression для многоклассовой 
# классификации по схеме one-vs-one через
# класс OneVsOneClassifier
lr_ovo_classifier.fit(X_otto_train, y_otto_train);

In [20]:
# вычисляем значения решающей функции 
# для тестовой выборки
lr_ovo_classifier_dec_func = np.round(
    lr_ovo_classifier.decision_function(X_otto_test), 3)
lr_ovo_classifier_dec_func[:5]

array([[ 0.724,  8.312,  7.295,  5.299,  4.233,  0.695,  6.283,  2.717,
         0.684],
       [ 4.279,  5.325,  6.322,  0.704, -0.33 ,  7.316,  8.325,  1.687,
         2.734],
       [ 0.688,  6.318,  5.313,  1.693, -0.328,  8.326,  7.321,  3.777,
         2.701],
       [ 3.289,  4.052,  2.698,  7.328, -0.33 ,  8.326,  3.774,  0.68 ,
         6.322],
       [ 1.672,  5.324,  6.33 ,  0.673,  1.669,  8.331,  4.321,  6.32 ,
         1.673]])

In [21]:
# вычисляем прогнозы для тестовой выборки
lr_ovo_classifier_pred = lr_ovo_classifier.predict(X_otto_test)
lr_ovo_classifier_pred[:5]

array([1, 6, 5, 5, 5])

In [22]:
# создаем игрушечные данные

# создаем обучающий массив признаков
X_trn = np.array([[4.2, 1.5],
                  [1.4, 2.1],
                  [3.1, 0.5],
                  [1.3, 2.2],
                  [6.9, 4.5],
                  [7.9, 7.1]])

# создаем обучающий массив меток
y_trn = np.array([0, 2, 0, 1, 2, 0])

# создаем тестовый массив признаков
X_tst = np.array([[2.8, 3.5],
                  [1.1, 1.8],
                  [8.9, 8.4]])

In [23]:
# создаем класс, выдающий константные прогнозы
class _ConstantPredictor():
    def fit(self, X, y):
        self.y_ = y
        return self

    def predict(self, X):
        return np.repeat(self.y_, X.shape[0])

    def decision_function(self, X):
        return np.repeat(self.y_, X.shape[0])

    def predict_proba(self, X):
        y_ = self.y_.astype(np.float64)
        return np.repeat([np.hstack([1 - y_, y_])], 
                         X.shape[0], axis=0)

In [24]:
# пишем функцию, которая обучает отдельный 
# бинарный классификатор
def _fit_binary(estimator, X, y, classes=None):
    """
    Обучает отдельный бинарный классификатор.
    """
    unique_y = np.unique(y)
    if len(unique_y) == 1:
        estimator = _ConstantPredictor().fit(X, unique_y)
    else:
        estimator = clone(estimator)
        estimator.fit(X, y)
    return estimator

In [25]:
# пишем функцию, которая обучает отдельный 
# бинарный классификатор по схеме one-vs-one
def _fit_ovo_binary(estimator, X, y, i, j, verbose):
    """
    Обучает отдельный бинарный 
    классификатор (one-vs-one).
    """
    cond = np.logical_or(y == i, y == j)
    y = y[cond]
    y_binary = np.empty(y.shape, int)
    y_binary[y == i] = 0
    y_binary[y == j] = 1
    indcond = np.arange(X.shape[0])[cond]
    
    if verbose:
        print(f"cравниваем класс {i} с классом {j}")
        print(f"индексы наблюдений, участвующих в сравнении:\n{indcond}\n")
    
    return (
        _fit_binary(
            estimator,
            _safe_split(estimator, X, None, indices=indcond)[0],
            y_binary,
            classes=[i, j],
        ),
        indcond,
    )

In [26]:
# пишем функцию, которая выдает прогнозы с
# помощью отдельного бинарного классификатора
def _predict_binary(estimator, X):
    """
    Выдает прогнозы с помощью отдельного 
    бинарного классификатора.
    """
    try:
        # значения решающей функции
        score = np.ravel(estimator.decision_function(X))
    except (AttributeError, NotImplementedError):
        # вероятности положительного класса
        score = estimator.predict_proba(X)[:, 1]
    return score

In [27]:
# пишем функцию, которая задает порог для
# прогнозов бинарного классификатора
def _threshold_for_binary_predict(estimator):
    """
    Задает порог для прогнозов 
    бинарного классификатора.
    """
    # если есть метод .decision_function()
    if hasattr(estimator, "decision_function"):
        return 0.0
    # в противном случае, т.е. если есть метод
    # .predict_proba()
    else:
        return 0.5

In [28]:
# пишем функцию, которая вычисляет итоговые уверенности на основе
# результатов многоклассовой классификации по схеме one-vs-one
def _ovr_decision_function(predictions, confidences, n_classes, verbose):
    """
    Вычисляет итоговые уверенности, исходя из результатов OvO.
    
    Параметры
    ----------
    predictions : массив формы (n_samples, n_classifiers)
        Классы, спрогнозированные каждым бинарным классификатором.
    confidences : массив формы (n_samples, n_classifiers)
        Значения решающей функции или спрогнозированные вероятности
        положительного класса, полученные с помощью каждого
        бинарного классификатора.
    n_classes : int
        Количество классов. n_classifiers должно быть
        ``n_classes * (n_classes - 1 ) / 2``.
    """
    n_samples = predictions.shape[0]
    votes = np.zeros((n_samples, n_classes))
    sum_of_confidences = np.zeros((n_samples, n_classes))
    
    if verbose:
        print(f"инициализируем матрицу голосов:\n{votes}\n")
        print(f"инициализируем матрицу сумм " 
              f"уверенностей:\n{sum_of_confidences}\n")

    k = 0
    for i in range(n_classes):
        for j in range(i + 1, n_classes):
            
            if verbose:
                print(f"сравниваем класс {i} с классом {j}\n")
            
            sum_of_confidences[:, i] -= confidences[:, k]
            sum_of_confidences[:, j] += confidences[:, k]
            votes[predictions[:, k] == 0, i] += 1
            votes[predictions[:, k] == 1, j] += 1
            k += 1
            
            if verbose:
                print(f"матрица голосов:\n{votes}\n")
                print(f"матрица сумм уверенностей:\n{sum_of_confidences}\n")

    # Выполняем монотонное преобразование сумм уверенностей в (-1/3, 1/3)
    # и потом добавим к голосам. Монотонное преобразование выглядит так:
    # f: x -> x / (3 * (|x| + 1)), используем 1/3 вместо 1/2, чтобы
    # не изменить порядок голосования. 
    # Мотивация состоит в том, чтобы использовать степени уверенности 
    # как способ разорвать связи в голосовании (ситуации, когда у
    # нескольких классов - одинаковое количество голосов), не изменяя
    # какое-либо решение на противоположное, исходя из разницы в
    # в 1 голос.
    transformed_confidences = sum_of_confidences / (
        3 * (np.abs(sum_of_confidences) + 1)
    )
    
    # получаем итоговые суммы уверенностей
    final_confidences = votes + transformed_confidences
    
    if verbose:
        print("подвергаем матрицу сумм уверенностей монотонному\n" 
              "преобразованию x / (3 * (|x| + 1))\n")
        print(f"матрица преобразованных сумм уверенностей:\n" 
              f"{transformed_confidences}\n")       
        print(f"матрица итоговых сумм уверенностей:\n" 
              f"{final_confidences}\n")
    
    return final_confidences

In [29]:
# реализуем собственный класс, выполняющий многоклассовую 
# классификацию по схеме one-vs-one
class CustomOneVsOneClassifier(MetaEstimatorMixin, 
                               ClassifierMixin, 
                               BaseEstimator):

    def __init__(self, estimator, n_jobs=None, verbose=False):
        self.estimator = estimator
        self.n_jobs = n_jobs
        self.verbose = verbose

    def fit(self, X, y):
        """
        Обучает соответствующие классификаторы.
        
        Параметры
        ----------
        X : массив формы (n_samples, n_features)
            Массив признаков.
        y : массив формы (n_samples,)
            Массив меток.
            
        Возвращает
        -------
        self : объект
            Обученная модель.
        """

        # записываем уникальные значения y
        self.classes_ = np.unique(y)
        
        if len(self.classes_) == 1:
            raise ValueError(
                "CustomOneVsOneClassifier нельзя обучить, "
                "если нет ни одного класса."
            )
        # записываем количество классов
        self.n_classes = self.classes_.shape[0]
        
        if self.verbose:
            print(f"количество классов: {self.n_classes}\n")
        
        # записываем список из двух кортежей, количество элементов
        # в кортежах определяется n_classes * (n_classes - 1 ) / 2
        # бинарных классификаторов, в первый кортеж записаны 
        # экземпляры моделей - бинарных классификаторов, во второй 
        # кортеж записаны массивы наблюдений, которые использовались 
        # для обучения соответствующего бинарного классификатора
        estimators_indices = list(
            zip(
                *(
                    Parallel(n_jobs=self.n_jobs)(
                        delayed(_fit_ovo_binary)(
                            self.estimator, X, y, self.classes_[i], 
                            self.classes_[j], self.verbose
                        )
                        for i in range(self.n_classes)
                        for j in range(i + 1, self.n_classes)
                    )
                )
            )
        )

        # записываем кортеж, состоящий из моделей 
        # - бинарных классификаторов
        self.estimators_ = estimators_indices[0]

        return self

    
    def predict(self, X):
        """
        Выдает метку класса с наибольшей уверенностью 
        для каждого наблюдения в массиве признаков X.
        Прогнозом будет ``argmax(decision_function(X), axis=1)``.
        
        Параметры
        ----------
        X : массив формы (n_samples, n_features)
            Массив признаков.
            
        Returns
        -------
        y : массив формы [n_samples]
            Массив спрогнозированных меток классов.
        """
        # получаем значения решающей функции
        Y = self.decision_function(X)
        # если 2 класса
        if self.n_classes == 2:
            # задаем порог
            thresh = _threshold_for_binary_predict(self.estimators_[0])
            # получаем прогнозы на основе порога
            pred = self.classes_[(Y > thresh).astype(int)]
            return pred
        pred = self.classes_[Y.argmax(axis=1)]
        if self.verbose:
            print(f"Прогнозом будет индекс с максимальной\n"
                  f"итоговой суммой уверенностей:\n{pred}")
        return pred

    def decision_function(self, X):
        """
        Решающая функция OneVsOneClassifier.
        Значения решающей функции для наблюдений вычисляются путем
        добавления к голосам нормализованной суммы уверенностей,
        вычисленных в результате попарных сравнений классов,
        чтобы устраненить неоднозначность, когда классы получают
        одинаковое количество голосов, образуя связь.
        
        Параметры
        ----------
        X : массив формы (n_samples, n_features)
            Массив признаков.
            
        Возвращает
        -------
        Y : массив формы (n_samples, n_classes) или (n_samples,)
            Результат вызова .decision_function() итоговой модели.
        """
        
        # получаем массив, состыкованный из len(self.estimators_)
        # массивов для получения прогнозов
        Xs = [X] * len(self.estimators_)
        
        # получаем прогнозы с помощью бинарных классификаторов
        predictions = np.vstack(
            [est.predict(Xi) for est, Xi in zip(self.estimators_, Xs)]
        ).T
        
        if self.verbose:
            print(f"матрица прогнозов бинарных " 
                  f"классификаторов:\n{predictions}\n")
        
        # получаем уверенности с помощью бинарных классификаторов
        confidences = np.vstack(
            [_predict_binary(est, Xi) 
             for est, Xi in zip(self.estimators_, Xs)]
        ).T
        
        if self.verbose:
            print(f"матрица уверенностей бинарных " 
                  f"классификаторов:\n{confidences}\n")
        
        # получаем итоговые уверенности
        Y = _ovr_decision_function(predictions, confidences, 
                                   len(self.classes_), self.verbose)
        # если 2 класса
        if self.n_classes == 2:
            return Y[:, 1]
        return Y


In [30]:
# строим логистическую регрессию по схеме one-vs-one
lr_ovo_cust_classifier = CustomOneVsOneClassifier(
    LogisticRegression(), verbose=True).fit(X_trn, y_trn)
# получим прогнозы для игрушечного 
# обучающего массива признаков
lr_ovo_cust_classifier_tr_pred = lr_ovo_cust_classifier.predict(X_trn)
lr_ovo_cust_classifier_tr_pred

количество классов: 3

cравниваем класс 0 с классом 1
индексы наблюдений, участвующих в сравнении:
[0 2 3 5]

cравниваем класс 0 с классом 2
индексы наблюдений, участвующих в сравнении:
[0 1 2 4 5]

cравниваем класс 1 с классом 2
индексы наблюдений, участвующих в сравнении:
[1 3 4]

матрица прогнозов бинарных классификаторов:
[[0 0 1]
 [1 1 1]
 [0 0 1]
 [1 1 1]
 [0 0 1]
 [0 0 1]]

матрица уверенностей бинарных классификаторов:
[[-2.04743668 -0.75518106  1.23741829]
 [ 0.54659607  0.47450729  0.14616936]
 [-1.38801103 -0.66553044  0.68037941]
 [ 0.66210856  0.54344122  0.11535615]
 [-3.50727061 -0.80156436  2.66134319]
 [-3.61460391 -0.34469341  3.34334993]]

инициализируем матрицу голосов:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

инициализируем матрицу сумм уверенностей:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

сравниваем класс 0 с классом 1

матрица голосов:
[[1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [

array([0, 2, 0, 2, 0, 0])

In [31]:
# получим прогнозы для игрушечного 
# тестового массива признаков
lr_ovo_cust_classifier_tst_pred = lr_ovo_cust_classifier.predict(X_tst)
lr_ovo_cust_classifier_tst_pred

матрица прогнозов бинарных классификаторов:
[[0 1 1]
 [1 1 0]
 [0 0 1]]

матрица уверенностей бинарных классификаторов:
[[-0.25562981  0.40092909  0.86834572]
 [ 0.71850161  0.49027405 -0.00858272]
 [-4.10030643 -0.30173175  3.8903464 ]]

инициализируем матрицу голосов:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

инициализируем матрицу сумм уверенностей:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

сравниваем класс 0 с классом 1

матрица голосов:
[[1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]

матрица сумм уверенностей:
[[ 0.25562981 -0.25562981  0.        ]
 [-0.71850161  0.71850161  0.        ]
 [ 4.10030643 -4.10030643  0.        ]]

сравниваем класс 0 с классом 2

матрица голосов:
[[1. 0. 1.]
 [0. 1. 1.]
 [2. 0. 0.]]

матрица сумм уверенностей:
[[-0.14529928 -0.25562981  0.40092909]
 [-1.20877566  0.71850161  0.49027405]
 [ 4.40203817 -4.10030643 -0.30173175]]

сравниваем класс 1 с классом 2

матрица голосов:
[[1. 0. 2.]
 [0. 2. 1.]
 [2. 0. 1.]]

матрица сумм уверенностей:
[[-0.14529928 -1.12397552  1.26

array([2, 1, 0])

In [32]:
# создаем экземпляр класса OutputCodeClassifier
lr_ecoc_classifier = OutputCodeClassifier(lr_pipe)
# применяем LogisticRegression для многоклассовой 
# классификации по схеме ECOC через
# класс OutputCodeClassifier
lr_ecoc_classifier.fit(X_otto_train, y_otto_train);

In [33]:
# вычисляем прогнозы для тестовой выборки
lr_ecoc_classifier_pred = lr_ecoc_classifier.predict(X_otto_test)
lr_ecoc_classifier_pred[:5]

array([1, 6, 5, 5, 5])

In [34]:
# реализуем собственный класс, выполняющий
# многоклассовую классификацию по схеме ECOC
class CustomOutputCodeClassifier(MetaEstimatorMixin, 
                                 ClassifierMixin, 
                                 BaseEstimator):
    """
    Класс, реализующий многоклассовую классификацию
    согласно подходу Error-Correcting Output Code
    
    Параметры
    ----------
    estimator : объект - экземпляр класса 
        Экземпляр класса, в котором реализована модель 
        машинного обучения. У него должен быть либо метод 
        .decision_function(), либо метод .predict_proba().
    code_size : float, значение по умолчанию 1.5
        Процент от количества классов для создания кодовой книги.
        Значение от 0 до 1 потребует меньшее количество 
        классификаторов, чем подход "один против остальных". 
        Значение выше 1 потребует большее число классификаторов,
        чем подход "один против остальных".
    metric: str, значение по умолчанию 'euclidean'
        Метрика расстояния.
    random_state : int, значение по умолчанию None
        Стартовое значение генератора псевдослучайных чисел.
    n_jobs : int, значение по умолчанию None
        Количество ядер процессора для распараллеливания.
    verbose: bool, значение по умолчанию False
        Печатает процесс обучения.
    
    Атрибуты
    ----------
    estimators_ : список `int(n_classes * code_size)` классификаторов
        Классификаторы, используемые для предсказания.
    classes_ : ndarray of shape (n_classes,)
        Массив с метками.
    code_book_ : ndarray формы (n_classes, code_size)
        Массив бинарных значений, содержащий код 
        каждого класса.
    """
    
    def __init__(self, estimator, code_size=1.5, metric='euclidean',
                 random_state=None, n_jobs=None, verbose=False):
        self.estimator = estimator
        self.code_size = code_size
        self.metric = metric
        self.random_state = random_state
        self.n_jobs = n_jobs
        self.verbose = verbose

    def fit(self, X, y):
        """
        Обучает классификаторы.
        
        Параметры
        ----------
        X : массив формы (n_samples, n_features)
            Массив признаков.
        y : массив формы (n_samples,)
            Массив меток.
            
        Возвращает
        -------
        self : object
            Обученная модель.
        """
        
        self.classes_ = np.unique(y)
        n_classes = self.classes_.shape[0]
        if n_classes == 0:
            raise ValueError(
                "CustomOutputCodeClassifier нельзя обучить, "
                "если нет ни одного класса.")
        
        code_size_ = int(n_classes * self.code_size)
        
        if self.verbose:
            print(f"количество классов: {n_classes}")
            print(f"code_size с поправкой на n_classes: {code_size_}") 
            print(f"размер кодовой книги n_classes x "
                  f"code_size:  {n_classes} x {code_size_}")
            print("")

        rng = np.random.RandomState(self.random_state)
        self.code_book_ = rng.uniform(size=(n_classes, code_size_))
        
        if self.verbose:
            print(f"исходная кодовая книга:\n{self.code_book_}")
            print("")
        
        self.code_book_[self.code_book_ > 0.5] = 1
        
        if self.verbose:
            print(f"первая корректировка кодовой книги:\n"
                  f"все значения больше 0.5 приравниваем к 1:\n"
                  f"{self.code_book_}\n")

        if hasattr(self.estimator, "decision_function"):
            self.code_book_[self.code_book_ != 1] = -1
        else:
            self.code_book_[self.code_book_ != 1] = 0
        
        if self.verbose:
            print(f"проверка наличия метода .decision_function(): "
                  f"{hasattr(self.estimator, 'decision_function')}\n")
            print(f"вторая корректировка кодовой книги:\n"
                  f"если есть метод .decision_function(), все значения,\n"
                  f"не являющиеся 1, приравниваем к -1\n"
                  f"если нет метода .decision_function(), все значения,\n"
                  f"не являющиеся 1, приравниваем к 0\n\n"
                  f"{self.code_book_}\n")
            
        classes_index = {c: i for i, c in enumerate(self.classes_)}
        
        if self.verbose:
            print(f"индексы классов:\n{classes_index}")
            print("")

        Y = np.array(
            [self.code_book_[classes_index[y[i]]] 
             for i in range(y.shape[0])],
            dtype=int,
        )

        self.estimators_ = Parallel(n_jobs=self.n_jobs)(
            delayed(_fit_binary)(self.estimator, X, Y[:, i]) 
            for i in range(Y.shape[1])
        )
        
        if self.verbose:    
            print(f"массив меток:\n{Y}\n")
            print("_ConstantPredictor() нужен для ситуаций, когда "
                  "столбец является константным")
            print("список обученных моделей:\n", self.estimators_)
            print("")

        return self
    
    def predict(self, X):
        """
        Получаем прогнозы с помощью соответствующих классификаторов.
        
        Параметры
        ----------
        X : массив формы (n_samples, n_features)
            Data.
        Массив признаков
        -------
        y : массив формы (n_samples,)
            Массив спрогнозированных меток классов.
        """
     
        Y = np.array([_predict_binary(e, X) 
                      for e in self.estimators_]).T
        dist = pairwise_distances(Y, self.code_book_,  
                                  metric=self.metric)
        pred = dist.argmin(axis=1)
           
        if self.verbose:
            print(f"прогнозы бинарных классификаторов:\n{Y}\n")
            print(f"расстояния:\n{dist}\n")
            print(f"итоговые прогнозы:\n{pred}")
            
        return self.classes_[pred]

In [35]:
# строим логистическую регрессию по схеме ECOC
lr_ecoc_cust_classifier = CustomOutputCodeClassifier(
    LogisticRegression(), 
    random_state=42, verbose=True)
lr_ecoc_cust_classifier.fit(X_trn, y_trn)
# получим прогнозы для игрушечного
# обучающего массива признаков
lr_ecoc_cust_classifier_tr_pred = lr_ecoc_cust_classifier.predict(X_trn)
lr_ecoc_cust_classifier_tr_pred

количество классов: 3
code_size с поправкой на n_classes: 4
размер кодовой книги n_classes x code_size:  3 x 4

исходная кодовая книга:
[[0.37454012 0.95071431 0.73199394 0.59865848]
 [0.15601864 0.15599452 0.05808361 0.86617615]
 [0.60111501 0.70807258 0.02058449 0.96990985]]

первая корректировка кодовой книги:
все значения больше 0.5 приравниваем к 1:
[[0.37454012 1.         1.         1.        ]
 [0.15601864 0.15599452 0.05808361 1.        ]
 [1.         1.         0.02058449 1.        ]]

проверка наличия метода .decision_function(): True

вторая корректировка кодовой книги:
если есть метод .decision_function(), все значения,
не являющиеся 1, приравниваем к -1
если нет метода .decision_function(), все значения,
не являющиеся 1, приравниваем к 0

[[-1.  1.  1.  1.]
 [-1. -1. -1.  1.]
 [ 1.  1. -1.  1.]]

индексы классов:
{0: 0, 1: 1, 2: 2}

массив меток:
[[-1  1  1  1]
 [ 1  1 -1  1]
 [-1  1  1  1]
 [-1 -1 -1  1]
 [ 1  1 -1  1]
 [-1  1  1  1]]

_ConstantPredictor() нужен для ситуа

array([0, 1, 0, 1, 0, 0])

In [36]:
# пишем функцию, вычисляющую евклидово расстояние
def euclidean_distance(x1, x2):
    """ 
    Вычисляет евклидово расстояние 
    между двумя векторами. 
    """
    distance = 0
    for i in range(len(x1)):
        distance += pow((x1[i] - x2[i]), 2)
    return np.sqrt(distance)

# кодовая книга
code_book = np.array([[-1.,  1.,  1.,  1.],
                      [-1., -1., -1.,  1.],
                      [ 1.,  1., -1.,  1.]])

# прогнозы бинарных классификаторов
binary_cl_pred = np.array(
    [[-0.95401785, 2.54618834,  0.63156985, 1.],
     [-0.57325232, 0.51581038, -1.16309398, 1.],
     [-1.00934633, 1.96254221,  0.44723263, 1.],
     [-0.54662428, 0.42968979, -1.2602978,  1.],
     [-0.72776042, 3.88431188,  0.85418853, 1.],
     [-0.39706322, 4.12206578,  0.30924707, 1.]]
)

# вычисляем расстояния
print(euclidean_distance(code_book[0], binary_cl_pred[0]))
print(euclidean_distance(code_book[0], binary_cl_pred[1]))
print(euclidean_distance(code_book[0], binary_cl_pred[2]))
print(euclidean_distance(code_book[0], binary_cl_pred[3]))
print(euclidean_distance(code_book[0], binary_cl_pred[4]))
print(euclidean_distance(code_book[0], binary_cl_pred[5]))
print("")
print(euclidean_distance(code_book[1], binary_cl_pred[0]))
print(euclidean_distance(code_book[1], binary_cl_pred[1]))
print(euclidean_distance(code_book[1], binary_cl_pred[2]))
print(euclidean_distance(code_book[1], binary_cl_pred[3]))
print(euclidean_distance(code_book[1], binary_cl_pred[4]))
print(euclidean_distance(code_book[1], binary_cl_pred[5]))
print("")
print(euclidean_distance(code_book[2], binary_cl_pred[0]))
print(euclidean_distance(code_book[2], binary_cl_pred[1]))
print(euclidean_distance(code_book[2], binary_cl_pred[2]))
print(euclidean_distance(code_book[2], binary_cl_pred[3]))
print(euclidean_distance(code_book[2], binary_cl_pred[4]))
print(euclidean_distance(code_book[2], binary_cl_pred[5]))

1.5901426087931865
2.2573277867455066
1.1100119932923562
2.3748156610146824
2.9007982340747644
3.2539156574237387

3.903791269560861
1.5831596056031012
3.2971542252756194
1.5222737545529064
5.231503802427298
5.321016679698285

2.9784063383459958
1.6541348484369
2.6567729419829744
1.6688485083435831
3.839586721876623
3.662404710791444


In [37]:
# получим прогнозы для игрушечного 
# тестового массива признаков
lr_ecoc_cust_classifier_tst_pred = lr_ecoc_cust_classifier.predict(X_tst)
lr_ecoc_cust_classifier_tst_pred

прогнозы бинарных классификаторов:
[[-0.48172898  1.2365915  -0.98211356  1.        ]
 [-0.59286446  0.36135728 -1.20187549  1.        ]
 [-0.28194127  4.5849551   0.31210383  1.        ]]

расстояния:
[[2.06236625 2.29592384 1.50060529]
 [2.32849298 1.43520267 1.72795701]
 [3.72031082 5.78177726 4.02702036]]

итоговые прогнозы:
[2 1 0]


array([2, 1, 0])